In [0]:
# Question 1
import pandas as pd 
import os

file_path = "/Volumes/workspace/default/assignment3"
city_files = { 
              "Austin": "a_listings.csv", 
              "Bangkok": "b_listings.csv", 
              "Buenos Aires": "Bu_listings.csv", 
              "Cape Town": "cape_listings.csv", 
              "Istanbul": "ista_listings.csv", 
              "Melbourne": "mel_listings.csv" } 
 
city_dfs = []
for city, fname in city_files.items():
    path = os.path.join(file_path, fname)
    df = pd.read_csv(path, low_memory=False)
    # normalize column names
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    df["city"] = city
    city_dfs.append(df)

In [0]:
#Question 2 Combine all city DataFrames into one 
all_listings = pd.concat(city_dfs, ignore_index=True) # Preview combined dataset 
print("\n📊 Combined dataset shape:", all_listings.shape) 
print(all_listings[['id', 'name', 'city', 'price']].head())


In [0]:
#Question 3
print(all_listings['city'].value_counts())





In [0]:
#Question 3
print(all_listings.head())

In [0]:
#Question 3
#all_listings['price'].describe()
all_listings[['price','minimum_nights','number_of_reviews']].describe()

In [0]:
#Question 4
df['price'] = df['price'].replace('[$,]', '', regex=True).astype(float)
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
df['last_review'] = df['last_review'].fillna(pd.NaT)
df = df.dropna(subset=['host_id', 'neighbourhood', 'room_type'])
print(df.info())
print(df.head())


In [0]:
#5. Top Hosts by Number of Listings
top_hosts = df.groupby('host_id').size().sort_values(ascending=False).head(10)
print(top_hosts)

In [0]:
#Question 6 : Price Distribution by City
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#print(all_listings['city'].unique())
price_stats = all_listings.groupby('city')['price'].agg(
    average_price='mean',
    median_price='median',
    min_price='min',
    max_price='max'
).reset_index()

print(" Price distribution by city:")
print(price_stats)

# Create a boxplot of price by city
plt.figure(figsize=(10,6))
sns.boxplot(data=all_listings, x='city', y='price')
plt.title('Price Distribution by City')
plt.xlabel('City')
plt.ylabel('Price ($)')
plt.xticks(rotation=45)
plt.grid(True, axis='y', linestyle='--', alpha=0.6)
plt.show()

In [0]:
#Question 7 : Availability Analysis
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
print("Availability Analysis:")
#print(all_listings.columns)
print(all_listings[['city', 'availability_365']].groupby('city').mean())    

In [0]:
#Question 8 :Neighborhood Insights For each city:

top_neighborhoods = all_listings.groupby('neighbourhood_cleansed')['id'].count().sort_values(ascending=False).head(3)
print(top_neighborhoods)
for neighborhood in top_neighborhoods.index:
    neighborhood_df = all_listings[all_listings['neighbourhood_cleansed'] == neighborhood]
    avg_price = neighborhood_df['price'].mean()
    avg_reviews = neighborhood_df['number_of_reviews'].mean()
    most_common_room_type = neighborhood_df['room_type'].mode()[0]
    print(f"Neighborhood: {neighborhood}")
    print(f"Average Price: ${avg_price:.2f}")
    print(f"Average Number of Reviews: {avg_reviews:.2f}")
    print(f"Most Common Room Type: {most_common_room_type}")
    print() 



In [0]:
#QUESTION 9 Room Type Trends 
room_type_stats = all_listings.groupby('room_type').agg(
    avg_price=('price', 'mean'),
    num_listings=('id', 'count'),
    avg_availability=('availability_365', 'mean')
)
print(room_type_stats)


In [0]:
#10. Review Trends Calculate the average number of reviews per listing, per city. Identify the top 5 listings (by id) with the highest number of reviews across all cities.
review_stats = all_listings.groupby('city')['number_of_reviews'].agg(
    avg_reviews_per_listing='mean'
)
print(review_stats)
top_listings = all_listings.groupby('id')['number_of_reviews'].sum().sort_values(ascending=False).head(5)
print(top_listings) 
